## Exercise 1: Evaluating NL2SQL Model with LLM-as-a-Judge & Analyzing Weaknesses in NL2SQL Tasks

### Step 1: Prepare Evaluation Dataset
- Collect evaluation data that includes:
  - Natural language questions
  - Gold SQL queries
  - Model-predicted SQL queries
- Dataset: We will use the **NL2SQL-Bugs** dataset which contains error annotations
- Purpose: To analyze **where LLMs fail to accurately evaluate SQL correctness**

---

### Step 2: LLM-as-a-Judge — SQL Text-Level Comparison
- Use LLM to compare the **text** of the gold and predicted SQL
- Ask the model:  
  **“Do these SQL queries express the same logic?”**
- Tasks:
  - Design prompt for textual comparison
  - Implement pipeline to invoke the LLM and collect judgments
- Evaluation Goal:
  - Determine whether the two SQL statements are logically equivalent (ignoring syntax)

| Comparison Target | Method | Requires DB Execution? | Notes |
|-------------------|--------|-------------------------|-------|
| SQL Text          | LLM Prompt Judgement | ❌ No | Sensitive to formatting, aliasing, keyword order |

- Error Analysis:
  - What types of SQL syntax variations confuse the LLM?
  - Are there consistent false positives or false negatives?

---

### Step 3: LLM-as-a-Judge — Execution Result Comparison
- Execute both gold and predicted SQL queries on the database
- Present the results as markdown tables (sorted, truncated)
- Ask the model:  
  **“Do these two result tables represent the same answer?”**
- Tasks:
  - Implement SQL execution on the same database
  - Format the results (row limit, sorted keys)
  - Design prompt for table comparison

| Comparison Target   | Method            | Requires DB Execution? | Notes |
|---------------------|-------------------|-------------------------|-------|
| Query Results       | LLM Table Comparison | ✅ Yes | Better for semantic equivalence but still prone to hallucination |

- Error Analysis:
  - What types of result mismatches are missed by the LLM?
  - Are formatting, ordering, or null values influencing LLM judgment?

---


# <Lets Start!>

### Step 1: Prepare Dataset

Import NL2SQL-Bugs-with-evidence.json from the **NL2SQLBugs** benchmark:
  - https://nl2sql-bugs.github.io/
  - Each example contains:
    - A natural language question
    - A gold (correct) SQL query
    - A predicted (incorrect) SQL query
    - Error evidence: explanation of why the prediction is incorrect
    - Error type: the category of the error (e.g., attribute error, join error)
    - and more...


> **Note:** In this example, we use an existing benchmark.
> In a real evaluation pipeline, you should construct a dataset that includes at least:
> - a natural language question,
> - the corresponding gold SQL query, and
> - a predicted SQL query generated by your model.



In [1]:
import pandas as pd
from azureml.fsspec import AzureMachineLearningFileSystem
from dotenv import load_dotenv
import os
import json

In [2]:
load_dotenv()

True

In [3]:
with open('../dataset/NL2SQL-Bugs-with-evidence.json') as f:
    data = json.load(f)

nl2sql_bug_df = pd.DataFrame(data)
nl2sql_bug_df["label"] = nl2sql_bug_df["label"].astype(bool)
len(nl2sql_bug_df.loc[nl2sql_bug_df["label"]== True])

1019

In [4]:
# -----------------------------------------------
# Check if the same natural language question has 
# multiple distinct gold (label=True) SQL queries.
# -----------------------------------------------

dupes = (
    nl2sql_bug_df[nl2sql_bug_df["label"] == True]
    .groupby("question")
    .filter(lambda x: len(x) > 1)
)

# Inspect duplicated gold SQLs
print(dupes)                         # All rows with duplicated questions
print(dupes["question"].unique())   # Unique question texts with multiple gold SQLs

        id                                           question  \
1126  1126  Show me the season page of year when the race ...   
1145  1145  Show me the season page of year when the race ...   

                                               evidence      db_id  \
1126                      race number refers to raceId;  formula_1   
1145  the season page refers to url; race number ref...  formula_1   

                                                    sql  label error_types  
1126  SELECT T2.url FROM races AS T1 INNER JOIN seas...   True          []  
1145  SELECT T2.url FROM races AS T1 INNER JOIN seas...   True          []  
['Show me the season page of year when the race No. 901 took place.']


Rearrange the NL2SQLBugs benchmark to build a new evaluation dataset:
  - Group multiple entries that share the same natural language question
  - For each question, create a triplet:
    - **Questions**(NL)
    - **DB_ID**
    - **Gold SQL** (correct)
    - **Predicted SQL (Incorrect)**
    - **Error Evidence**
    - **Error Types**

In [5]:
# -----------------------------------------------
# Create a mapping from each question to its gold SQL query.
# Assumes that each question has at most one correct (label=True) SQL.
# -----------------------------------------------

# Ensure the 'label' column is boolean
nl2sql_bug_df["label"] = nl2sql_bug_df["label"].astype(bool)

# Build a mapping: question → gold_sql
gold_map = (
    nl2sql_bug_df
    .loc[nl2sql_bug_df["label"]]         # Keep only correct examples
    .set_index("question")["sql"]        # Set 'question' as index and select 'sql'
    .to_dict()                           # Convert to dictionary
)

# Check how many unique gold questions exist
len(gold_map)


1018

In [6]:
# -----------------------------------------------
# Construct evaluation pairs by collecting all incorrect (label=False) predictions,
# and attaching the corresponding gold SQL (if available) for comparison.
# -----------------------------------------------

eval_pipeline_test_list = []

for _, row in nl2sql_bug_df.loc[~nl2sql_bug_df["label"]].iterrows():
    question = row["question"]
    pred_sql = row["sql"]
    
    eval_pipeline_test_list.append({
        "question":    question,
        "db_id":       row["db_id"],
        "gold_sql":    gold_map.get(question),  # None if no correct SQL exists for this question
        "pred_sql":    pred_sql,
        "label":       False,
        "evidence":    row["evidence"],
        "error_types": row["error_types"],
    })


In [7]:
# -----------------------------------------------
# Filter out examples that do not have a gold SQL available,
# so that only valid (gold, predicted) SQL pairs remain for evaluation.
# -----------------------------------------------
eval_pipeline_test_df = pd.DataFrame(eval_pipeline_test_list)
eval_pipeline_test_df["gold_sql"].isnull().sum()
eval_pipeline_test_df = eval_pipeline_test_df.loc[eval_pipeline_test_df["gold_sql"].notnull()].copy()

In [8]:
#Final test set with 1) Question 2) db_id 3) gold_sql 4) pred_sql 5) label
eval_pipeline_test_df

,question,db_id,gold_sql,pred_sql,label,evidence,error_types
0,What is the highest eligible free rate for K-1...,california_schools,SELECT MAX(CAST(`Free Meal Count (K-12)` AS RE...,SELECT max(`free meal count (k-12)`) / max(`en...,False,Eligible free rate for K-12 = `Free Meal Count...,"[{'error_type': 'Function-Related Errors', 'su..."
1,Please list the lowest three eligible free rat...,california_schools,SELECT T2.`Free Meal Count (Ages 5-17)` / T2.`...,SELECT min(`free meal count (ages 5-17)`) FROM...,False,Eligible free rates for students aged 5-17 = `...,"[{'error_type': 'Attribute-Related Errors', 's..."
2,Please list the zip code of all the charter sc...,california_schools,SELECT T2.Zip FROM frpm AS T1 INNER JOIN schoo...,SELECT schools.zip FROM schools INNER JOIN frp...,False,Charter schools refers to `Charter School (Y/N...,"[{'error_type': 'Value-Related Errors', 'sub_e..."
3,What is the unabbreviated mailing street addre...,california_schools,SELECT T2.MailStreet FROM frpm AS T1 INNER JOI...,SELECT schools.streetabr FROM frpm INNER JOIN ...,False,,"[{'error_type': 'Attribute-Related Errors', 's..."
8,Among the schools with the SAT test takers of ...,california_schools,SELECT T2.School FROM satscores AS T1 INNER JO...,SELECT schools.school FROM schools INNER JOIN ...,False,Magnet schools or offer a magnet program means...,"[{'error_type': 'Attribute-Related Errors', 's..."
...,...,...,...,...,...,...,...
986,For all the transactions happened during 8:00-...,debit_card_specializing,SELECT count(transactions_1k.transactionid) FR...,select count(transactions_1k.transactionid) fr...,False,Czech Republic can be represented as the Count...,"[{'error_type': 'Attribute-Related Errors', 's..."
987,There's one customer spent 214582.17 in the Ju...,debit_card_specializing,SELECT customers.currency FROM customers INNER...,select distinct customers.currency from custom...,False,June of 2013 means Date contains '201306' in t...,"[{'error_type': 'Value-Related Errors', 'sub_e..."
993,Which gas station has the highest amount of re...,debit_card_specializing,SELECT gasstationid FROM transactions_1k GROUP...,select gasstations.gasstationid from gasstatio...,False,,"[{'error_type': 'Table-Related Errors', 'sub_e..."
997,For all the people who paid more than 29.00 pe...,debit_card_specializing,SELECT T1.Consumption FROM yearmonth AS T1 INN...,select distinct yearmonth.consumption from yea...,False,August of 2012 refers to the Date value = '201...,"[{'error_type': 'Value-Related Errors', 'sub_e..."


### Step 2: LLM-as-a-Judge — Raw SQL Comparison

- We use an LLM-as-a-Judge evaluator that directly compares the gold SQL and the predicted SQL to determine whether the model's output is correct.
- The evaluation logic, including the prompt and implementation, is located in the `evaluation/evaluators/` folder.


In [9]:
import sys
import os
from evaluators.llm_as_judge_raw_sql_evaluator import LLMasJudgeRawSQL
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

# Load AzureOpenAIModel
model_config_4o = AzureOpenAIModelConfiguration(
    azure_endpoint= os.environ["AZURE_ENDPOINT"],
    azure_key = os.environ["AZURE_API_KEY"],
    azure_deployment = os.environ["AZURE_4O_DEPLOYMENT"],
    api_version = os.environ["AZURE_4O_API_VERSION"]
)

model_config_o4_mini = AzureOpenAIModelConfiguration(
    azure_endpoint= os.environ["AZURE_ENDPOINT"],
    azure_key = os.environ["AZURE_API_KEY"],
    azure_deployment = os.environ["AZURE_O4_MINI_DEPLOYMENT"],
    api_version = os.environ["AZURE_O4_MINI_API_VERSION"]
)


/anaconda/envs/nl2sql_ssfn/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


[INFO] Could not import AIAgentConverter. Please install the dependency with `pip install azure-ai-projects`.
[INFO] Could not import SKAgentConverter. Please install the dependency with `pip install semantic-kernel`.


In [ ]:
import json
from tqdm import tqdm

# Initialize the LLM-based evaluator for raw SQL comparison
llm_sql_evaluator = LLMasJudgeRawSQL(model_config=model_config_o4_mini)
llm_raw_sql_pipeline_eval_result = []

# Loop through each evaluation example
for i, row in tqdm(eval_pipeline_test_df.iterrows(), ncols=100, colour="cyan", total=len(eval_pipeline_test_df)):
    question = row["question"]
    gold_sql = row["gold_sql"]
    pred_sql = row["pred_sql"]

    try:
        # Get response from LLM evaluator
        result = llm_sql_evaluator(question=question, gold_sql=gold_sql, pred_sql=pred_sql)
    
        # Convert to dict if still string
        if isinstance(result, str):
            result = json.loads(result)
        elif not isinstance(result, dict):
            raise TypeError(f"Unexpected result type: {type(result)}")
    
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "llm_judgement": result["label"],
            "reason": result["reason"]
        })
    
        print(f"===== Row {i} Debug Log =====")
        print("Question:", question)
        print("Gold SQL:", gold_sql)
        print("Predicted SQL:", pred_sql)
        print("LLM Judgement:", result["label"])
        print("Reason:", result["reason"])
    
    
    except json.JSONDecodeError as e:
        # Handle cases where LLM output is not valid JSON
        print(f"JSONDecodeError at row {i}: {e}")
        print("Raw response was:")
        print(repr(result))  # Use repr to inspect escape characters and formatting issues

        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "llm_judgement": "ERROR",
            "reason": f"JSONDecodeError: {str(e)}"
        })

    except Exception as e:
        # Catch-all for other unexpected exceptions
        print(f"Unexpected error at row {i}: {e}")
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "llm_judgement": "ERROR",
            "reason": f"Exception: {str(e)}"
        })





  0%|                                                                       | 0/549 [00:00<?, ?it/s]

  0%|                                                             | 1/549 [00:11<1:41:27, 11.11s/it]

===== Row 0 Debug Log =====
Question: What is the highest eligible free rate for K-12 students in the schools in Alameda County?
Gold SQL: SELECT MAX(CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) FROM frpm WHERE `County Name` = 'Alameda'
Predicted SQL: SELECT max(`free meal count (k-12)`) / max(`enrollment (k-12)`) FROM frpm WHERE `county name` = 'Alameda'
LLM Judgement: incorrect
Reason: The predicted query divides the maximum free meal count by the maximum enrollment, which is not equivalent to selecting the maximum per-school free rate as required by the question.


  0%|▏                                                            | 2/549 [00:17<1:13:18,  8.04s/it]

===== Row 1 Debug Log =====
Question: Please list the lowest three eligible free rates for students aged 5-17 in continuation schools.
Gold SQL: SELECT T2.`Free Meal Count (Ages 5-17)` / T2.`Enrollment (Ages 5-17)` AS EligibleFreeRate FROM schools AS T1 INNER JOIN frpm AS T2 ON T1.CDSCode = T2.CDSCode WHERE T1.EdOpsName = 'Continuation School' AND T2.`Enrollment (Ages 5-17)` > 0 AND T2.`Free Meal Count (Ages 5-17)` IS NOT NULL AND T2.`Enrollment (Ages 5-17)` IS NOT NULL ORDER BY EligibleFreeRate ASC LIMIT 3
Predicted SQL: SELECT min(`free meal count (ages 5-17)`) FROM frpm WHERE `educational option type` = 'Continuation School'
LLM Judgement: incorrect
Reason: The predicted query only computes the minimum free meal count without dividing by enrollment to get the rate, omits the join to schools, and does not return the lowest three rates as required.


...

===== Row 651 Debug Log =====
Question: Please calculate the race completion percentage of Japanese drivers from 2007 to 2009.
Gold SQL: SELECT cast(sum(CASE WHEN results.time IS NOT NULL THEN 1 ELSE 0 END) AS REAL) * 100 / count(results.driverid) FROM results INNER JOIN races ON results.raceid = races.raceid INNER JOIN drivers ON results.driverid = drivers.driverid WHERE drivers.nationality = 'Japanese' AND races.year BETWEEN 2007 AND 2009
Predicted SQL: select cast(sum(case when races.time is not null then 1 else 0 end) as real) * 100 / count(drivers.driverid) from results inner join drivers on results.driverid = drivers.driverid inner join races on results.raceid = races.raceid where races.year between 2007 and 2009
LLM Judgement: incorrect
Reason: The predicted SQL omits the nationality filter for Japanese drivers and incorrectly references races.time instead of results.time, so it doesn’t compute the intended completion percentage for Japanese drivers.


 65%|███████████████████████████████████████▌                     | 356/549 [42:42<23:20,  7.26s/it]

===== Row 652 Debug Log =====
Question: Please calculate the race completion percentage of Japanese drivers from 2007 to 2009.
Gold SQL: SELECT cast(sum(CASE WHEN results.time IS NOT NULL THEN 1 ELSE 0 END) AS REAL) * 100 / count(results.driverid) FROM results INNER JOIN races ON results.raceid = races.raceid INNER JOIN drivers ON results.driverid = drivers.driverid WHERE drivers.nationality = 'Japanese' AND races.year BETWEEN 2007 AND 2009
Predicted SQL: SELECT (CAST(SUM(CASE WHEN T2.time IS NOT NULL THEN 1 ELSE 0 END) AS REAL) / COUNT(*)) * 100 AS completion_percentage FROM races T1 INNER JOIN results T2 ON T1.raceId = T2.raceId INNER JOIN drivers T3 ON T2.driverId = T3.driverId WHERE T3.nationality = 'Japanese' AND T1.year BETWEEN 2007 AND 2009
LLM Judgement: correct
Reason: The predicted query applies the same filters and computes the percentage by dividing the count of non‐null times by the total count (using COUNT(*) equivalent to COUNT(driverId)), and then multiplies by 100, yie

 65%|███████████████████████████████████████▋                     | 357/549 [42:48<21:21,  6.67s/it]

===== Row 656 Debug Log =====
Question: Which drivers born after 1975 have been ranked 2? Please give their forenames and surnames.
Gold SQL: select drivers.forename , drivers.surname from results inner join drivers on results.driverid = drivers.driverid where results.rank = 2 and strftime('%Y' , drivers.dob) > '1975'
Predicted SQL: SELECT drivers.forename, drivers.surname FROM drivers INNER JOIN results ON drivers.driverid = results.driverid WHERE strftime('%Y', drivers.dob) > '1975' AND results.position = 2
LLM Judgement: incorrect
Reason: The predicted query filters on results.position instead of results.rank, so it does not apply the same rank=2 condition as the gold query.


### Evaluation Analysis for Step 2: Can the LLM Identify Incorrect SQL Predictions?
- This evaluation tests the LLM-as-a-Judge on predicted SQLs known to be wrong (label=False).

- What we measure
    - False Positives (FP): LLM says "correct" for a wrong SQL
    - True Negatives (TN): LLM correctly says "incorrect" for a wrong SQL

> Why only FP and TN?: The dataset contains only incorrect predictions

We are testing the LLM’s ability to catch errors, not verify correct SQLs

In [ ]:
llm_raw_sql_pipeline_eval_df = pd.DataFrame(llm_raw_sql_pipeline_eval_result)

#Join with Original Eval test set 

merged_llm_raw_sql_pipeline_eval_df = pd.merge(eval_pipeline_test_df, llm_raw_sql_pipeline_eval_df, on="pred_sql", how='left')
merged_llm_raw_sql_pipeline_eval_df.drop(["question_y","gold_sql_y"], axis = 1)

fp = len(merged_llm_raw_sql_pipeline_eval_df[merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "correct"])
tn = len(merged_llm_raw_sql_pipeline_eval_df[merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "incorrect"])


print("Total:", len(merged_llm_raw_sql_pipeline_eval_df))
print("FP:", fp)
print("TN:", tn)
print(fp/tn*100)


Total: 549
FP: 0
TN: 2
0.0


### Error Type Analysis Part 2: LLM- as-a-Judge SQL Comparison  
> What SQL Prediction Errors Does the LLM Fail to Detect?

> This section investigates **which types of SQL errors are missed by the LLM**, specifically in cases where it incorrectly judges an incorrect SQL as correct (False Positives).

---

#### What We Measure

- **Error Types Missed by the LLM (False Positives)**  
  We filter cases where:
  - The predicted SQL is wrong (`label = False`)
  - The LLM incorrectly judges it as **"correct"**  
  → These are the **False Positives**, and we analyze their error types.

- **Total Distribution of Error Types**  
  We also calculate how often each error type appears across **all incorrect predictions**.  
  This gives context for how frequently each type occurs overall.

---

By comparing the two distributions, we can identify:
- Which error types are **frequently missed** by the LLM
- Whether certain rare or subtle errors tend to **slip past** the LLM's judgment


In [ ]:
# Print the error types associated with all false positive predictions
print(
    merged_llm_raw_sql_pipeline_eval_df[
        merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "correct"
    ]["error_types"]
)

# Count the total occurrences of each (error_type, sub_error_type) across all examples
total_errortypes_count = {}
for _, row in merged_llm_raw_sql_pipeline_eval_df.iterrows():
    error_type = row["error_types"][0]["error_type"]
    sub_error_type = row["error_types"][0]["sub_error_type"]
    key_tuple = (error_type, sub_error_type)
    total_errortypes_count[key_tuple] = total_errortypes_count.get(key_tuple, 0) + 1

# Show the overall error type distribution
print(total_errortypes_count)

# Count how often each (error_type, sub_error_type) appears in false positives
fp_errortypes_count = {}
for _, row in merged_llm_raw_sql_pipeline_eval_df[
    merged_llm_raw_sql_pipeline_eval_df["llm_judgement"] == "correct"
].iterrows():
    error_type = row["error_types"][0]["error_type"]
    sub_error_type = row["error_types"][0]["sub_error_type"]
    key_tuple = (error_type, sub_error_type)
    fp_errortypes_count[key_tuple] = fp_errortypes_count.get(key_tuple, 0) + 1

# Show what types of errors the LLM tends to miss
print(fp_errortypes_count)




In [ ]:
# ----------------------------------------
# Objective: Visualize and compare error type distributions
# between all predictions and false positives (cases judged correct by the LLM),
# using both absolute counts and relative frequencies.
# ----------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assume these dictionaries are already defined:
# - total_errortypes_count: error counts from all predictions
# - fp_errortypes_count: error counts only from false positives (LLM said "correct")

# ========================================
# Analysis: Visualize Error Distributions by Absolute Count
# ========================================

# Collect all unique error type keys and fill missing values with 0
all_keys = sorted(set(total_errortypes_count.keys()).union(fp_errortypes_count.keys()))
full_dist1 = {key: total_errortypes_count.get(key, 0) for key in all_keys}
full_dist2 = {key: fp_errortypes_count.get(key, 0) for key in all_keys}

# Create DataFrames for both distributions
df1 = pd.DataFrame(list(full_dist1.items()), columns=["Error Type", "Count"])
df1["Distribution"] = "All Predictions"

df2 = pd.DataFrame(list(full_dist2.items()), columns=["Error Type", "Count"])
df2["Distribution"] = "False Positives"

# Split (error_type, sub_error_type) tuple into separate columns
df1[["Error Category", "Error Subtype"]] = pd.DataFrame(df1["Error Type"].tolist(), index=df1.index)
df2[["Error Category", "Error Subtype"]] = pd.DataFrame(df2["Error Type"].tolist(), index=df2.index)

# Combine both DataFrames for plotting
combined_df = pd.concat([df1, df2])

# Determine plotting order based on total counts across both distributions
error_order = combined_df.groupby("Error Subtype")["Count"].sum().sort_values(ascending=False).index

# Plot absolute error counts
plt.figure(figsize=(16, 10))
sns.set_theme(style="whitegrid")
sns.barplot(
    x="Count",
    y="Error Subtype",
    hue="Distribution",
    data=combined_df,
    order=error_order,
    palette="viridis"
)
plt.title("Comparison of Error Distributions", fontsize=16)
plt.xlabel("Number of Errors", fontsize=12)
plt.ylabel("Error Subtype", fontsize=12)
plt.legend(title="Distribution", fontsize=10, title_fontsize=12)
plt.tight_layout()
plt.show()

# ========================================
# Analysis 2: Visualize Error Distributions by Relative Frequency (%)
# ========================================

# Calculate relative frequencies (%) for each distribution
df1["Relative Frequency (%)"] = (df1["Count"] / df1["Count"].sum()) * 100
df2["Relative Frequency (%)"] = (df2["Count"] / df2["Count"].sum()) * 100

# Re-split error type columns if necessary (defensive copy)
df1[["Error Category", "Error Subtype"]] = pd.DataFrame(df1["Error Type"].tolist(), index=df1.index)
df2[["Error Category", "Error Subtype"]] = pd.Da


SyntaxError: invalid character '—' (U+2014) (2272596185.py, line 4)

### Step 3: LLM-as-a-Judge — Execution Result Comparison
- In this section we will implement LLM-as-a-Judge evaluator that comapres actual exection result of gold and pred sql by querying to database. 
- This method is more widely used in recent evaluation of Text to SQL paper than driect comaprison of raw sql. 
- This section we will implement exection match by LLM but we will also implement this in rule-based in next exercise


> The evaluation logic, including the prompt and implementation, is located in the `evaluation/evaluators/` folder.  
> Also, all the util functions including formating, querying database and printing markdown is in the `evaluation/db_utils`

In [ ]:
import os 
import pandas as pd 
from db_utils.db_utils import get_db_path, execute_query, format_results_for_llm

# Set the base directory to the project root
base_dir = os.path.abspath("..")


# Make a copy of the evaluation DataFrame for execution matching
exec_match_test_df = eval_pipeline_test_df.copy()

# Add columns to store execution results for gold and predicted SQL
exec_match_test_df["gold_result"] = None
exec_match_test_df["pred_result"] = None


NameError: name 'eval_pipeline_test_df' is not defined

In [ ]:
# Iterate through each row to execute queries and format results
for idx, row in exec_match_test_df.iterrows():
    db_id    = row["db_id"]
    gold_sql = row["gold_sql"]
    pred_sql = row["pred_sql"]

    try:
        # Get the full path to the database file
        db_path = get_db_path(base_dir, db_id)
    except FileNotFoundError as e:
        # If the database file is not found, record the error and skip
        exec_match_test_df.at[idx, "gold_result"] = f"ERROR: {e}"
        exec_match_test_df.at[idx, "pred_result"] = f"ERROR: {e}"
        continue

    # Execute the gold SQL and format the result
    gold_raw = execute_query(db_path, gold_sql)
    exec_match_test_df.at[idx, "gold_result"] = format_results_for_llm(
        gold_raw,
        sort_keys=list(gold_raw[0].keys()) if gold_raw else None,
        row_limit=10  # Limit rows for readability
    )

    # Execute the predicted SQL and format the result
    pred_raw = execute_query(db_path, pred_sql)
    exec_match_test_df.at[idx, "pred_result"] = format_results_for_llm(
        pred_raw,
        sort_keys=list(pred_raw[0].keys()) if pred_raw else None,
        row_limit=5  # Limit rows for readability
    )


NameError: name 'exec_match_test_df' is not defined

In [ ]:
from db_utils.db_utils import print_markdown_table

# Check a sample of the execution results
exec_match_test_df[["db_id", "gold_result", "pred_result"]].head(10)



In [ ]:
# Display formatted results for a few sample rows
for idx in range(3):
    print(f"\n====== ✅ Row {idx} / DB: {exec_match_test_df.iloc[idx]['db_id']} ======\n")
    
    # Show the gold query result in markdown table format
    print_markdown_table(
        md_str=exec_match_test_df.iloc[idx]["gold_result"],
        title="GOLD RESULT"
    )
    
    # Show the predicted query result in markdown table format
    print_markdown_table(
        md_str=exec_match_test_df.iloc[idx]["pred_result"],
        title="PREDICTED RESULT"
    )



In [ ]:
from evaluators.llm_as_judge_exec_match_evaluator import LLMasJudgeExecMatch

###############################################################
# Objective: Use an LLM-based evaluator to compare the gold and predicted execution results
# and collect judgments on whether the predictions are semantically equivalent.
###############################################################

llm_exec_match_evaluator = LLMasJudgeExecMatch(model_config=model_config_o4_mini)
llm_exec_match_pipeline_eval_result = []

# Iterate through the test set and evaluate each row using the LLM
for i, row in tqdm(exec_match_test_df.iterrows(), ncols=100, colour="cyan", total=len(eval_pipeline_test_df)):
    question = row["question"]
    gold_result = row["gold_result"]
    pred_result = row["pred_result"]

    try:
        # Call the evaluator to get LLM's judgment
        result = llm_exec_match_evaluator(question=question, gold_result=gold_result, pred_result=pred_result)
        result = json.loads(result)  # Parse JSON response

        # Store the evaluation result
        llm_exec_match_pipeline_eval_result.append({
            "question": question,
            "gold_result": gold_result,
            "pred_result": pred_result,
            "llm_judgement": result["label"],
            "reason": result["reason"]
        })

        # Debug log
        print(f"##### {i}th DEBUG LOG #####")
        print("Question:", question)
        print("Gold Result:", print_markdown_table(gold_result))
        print("Pred Result:", print_markdown_table(pred_result))
        print("LLM Judgement:", result["label"])
        print("Reason:", result["reason"])

    except json.JSONDecodeError as e:
        # Handle case when LLM output is not valid JSON
        print(f"JSONDecodeError at row {i}: {e}")
        print("Raw response was:")
        print(repr(result))  # Print raw output for debugging
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_result": gold_result,
            "pred_result": pred_result,
            "llm_judgement": "ERROR",
            "reason": f"JSONDecodeError: {str(e)}"
        })

    except Exception as e:
        # Handle any other unexpected error
        print(f"Unexpected error at row {i}: {e}")
        llm_raw_sql_pipeline_eval_result.append({
            "question": question,
            "gold_result": gold_result,
            "pred_result": pred_result,
            "llm_judgement": "ERROR",
            "reason": f"Exception: {str(e)}"
        })


### Evaluation Analysis: LLM-as-a-Judge Execution Comparison  
> Can the LLM Identify Incorrect SQL Predictions?
- This evaluation tests the LLM-as-a-Judge on predicted SQLs known to be wrong (label=False).

- What we measure
    - False Positives (FP): LLM says "correct" for a wrong SQL
    - True Negatives (TN): LLM correctly says "incorrect" for a wrong SQL

> Why only FP and TN?: The dataset contains only incorrect predictions

- We are testing the LLM’s ability to catch errors, not verify correct SQLs

In [ ]:
#######################################################
# Objective: Aggregate and evaluate LLM judgment results by counting how many predictions were marked
# as correct or incorrect, and calculate the ratio between them.
#######################################################

# Convert evaluation results to a DataFrame
llm_exec_match_pipeline_eval_df = pd.DataFrame(llm_exec_match_pipeline_eval_result)

# Count how many predictions were judged as "correct" by the LLM (i.e., false positives)
fp = len(llm_exec_match_pipeline_eval_df[llm_exec_match_pipeline_eval_df["llm_judgement"] == "correct"])

# Count how many predictions were judged as "incorrect" by the LLM (i.e., true negatives)
tn = len(llm_exec_match_pipeline_eval_df[llm_exec_match_pipeline_eval_df["llm_judgement"] == "incorrect"])

# Print summary statistics
print("Total:", len(llm_exec_match_pipeline_eval_df))
print("FP:", fp)
print("TN:", tn)

# Calculate and print the false positive to true negative ratio in percentage
print(fp / tn * 100)

NameError: name 'llm_exec_match_pipeline_eval_result' is not defined

### Error Type Analysis Part 2: LLM- as-a-Judge Execution Result Comparison  
> What SQL Prediction Errors doews LLM Fail to Detect?

> This section investigates **which types of SQL errors are missed by the LLM**, specifically in cases where it incorrectly judges an incorrect SQL as correct (False Positives).

---

#### What We Measure

- **Error Types Missed by the LLM (False Positives)**  
  We filter cases where:
  - The predicted SQL is wrong (`label = False`)
  - The LLM incorrectly judges it as **"correct"**  
  → These are the **False Positives**, and we analyze their error types.

- **Total Distribution of Error Types**  
  We also calculate how often each error type appears across **all incorrect predictions**.  
  This gives context for how frequently each type occurs overall.

---

By comparing the two distributions, we can identify:
- Which error types are **frequently missed** by the LLM
- Whether certain rare or subtle errors tend to **slip past** the LLM's judgment

In [ ]:
#Join with Original Eval test set 
merged_llm_exec_match_pipeline_eval_df = pd.merge(exec_match_test_df, llm_exec_match_pipeline_eval_df, on="pred_result", how='left')
merged_llm_exec_match_pipeline_eval_df.drop(["question_y","gold_result_y"], axis = 1)

In [ ]:
#######################################################################
# Objective: Count the frequency of each (error_type, sub_error_type) pair across all examples,
# and separately count them for cases where the LLM incorrectly judged the prediction as correct (false positives).
#######################################################################

# Dictionary to store total counts of (error_type, sub_error_type) across all rows
total_errortypes_count = {}
for _, row in merged_llm_exec_match_pipeline_eval_df.iterrows():
    error_types = [x["error_type"] for x in row["error_types"]]
    sub_error_types = [x["sub_error_type"] for x in row["error_types"]]
    key_tuples = zip(error_types, sub_error_types)
    for key_tuple in key_tuples:
        total_errortypes_count[key_tuple] = total_errortypes_count.get(key_tuple, 0) + 1

# Print total error type distribution
print(total_errortypes_count)

# Dictionary to store counts of (error_type, sub_error_type) only for false positives
fp_errortypes_count = {}
for _, row in merged_llm_exec_match_pipeline_eval_df[
    merged_llm_exec_match_pipeline_eval_df["llm_judgement"] == "correct"
].iterrows():
    error_types = [x["error_type"] for x in row["error_types"]]
    sub_error_types = [x["sub_error_type"] for x in row["error_types"]]
    key_tuples = zip(error_types, sub_error_types)
    for key_tuple in key_tuples:
        fp_errortypes_count[key_tuple] = fp_errortypes_count.get(key_tuple, 0) + 1

# Print false positive error type distribution
print(fp_errortypes_count)



In [ ]:
# ----------------------------------------
# Objective: Visualize and compare error type distributions
# between all predictions and false positives (cases judged correct by the LLM),
# using both absolute counts and relative frequencies.
# ----------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assume these dictionaries are already defined:
# - total_errortypes_count: error counts from all predictions
# - fp_errortypes_count: error counts only from false positives (LLM said "correct")

# ========================================
# Analysis: Visualize Error Distributions by Absolute Count
# ========================================

# Collect all unique error type keys and fill missing values with 0
all_keys = sorted(set(total_errortypes_count.keys()).union(fp_errortypes_count.keys()))
full_dist1 = {key: total_errortypes_count.get(key, 0) for key in all_keys}
full_dist2 = {key: fp_errortypes_count.get(key, 0) for key in all_keys}

# Create DataFrames for both distributions
df1 = pd.DataFrame(list(full_dist1.items()), columns=["Error Type", "Count"])
df1["Distribution"] = "All Predictions"

df2 = pd.DataFrame(list(full_dist2.items()), columns=["Error Type", "Count"])
df2["Distribution"] = "False Positives"

# Split (error_type, sub_error_type) tuple into separate columns
df1[["Error Category", "Error Subtype"]] = pd.DataFrame(df1["Error Type"].tolist(), index=df1.index)
df2[["Error Category", "Error Subtype"]] = pd.DataFrame(df2["Error Type"].tolist(), index=df2.index)

# Combine both DataFrames for plotting
combined_df = pd.concat([df1, df2])

# Determine plotting order based on total counts across both distributions
error_order = combined_df.groupby("Error Subtype")["Count"].sum().sort_values(ascending=False).index

# Plot absolute error counts
plt.figure(figsize=(16, 10))
sns.set_theme(style="whitegrid")
sns.barplot(
    x="Count",
    y="Error Subtype",
    hue="Distribution",
    data=combined_df,
    order=error_order,
    palette="viridis"
)
plt.title("Comparison of Error Distributions", fontsize=16)
plt.xlabel("Number of Errors", fontsize=12)
plt.ylabel("Error Subtype", fontsize=12)
plt.legend(title="Distribution", fontsize=10, title_fontsize=12)
plt.tight_layout()
plt.show()

# ========================================
# Analysis 2: Visualize Error Distributions by Relative Frequency (%)
# ========================================

# Calculate relative frequencies (%) for each distribution
df1["Relative Frequency (%)"] = (df1["Count"] / df1["Count"].sum()) * 100
df2["Relative Frequency (%)"] = (df2["Count"] / df2["Count"].sum()) * 100

# Re-split error type columns if necessary (defensive copy)
df1[["Error Category", "Error Subtype"]] = pd.DataFrame(df1["Error Type"].tolist(), index=df1.index)
df2[["Error Category", "Error Subtype"]] = pd.Da

